In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 15
loss_function = BinaryCrossentropy()
max_sequence_length = 300
num_distinct_words = 5000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

In [3]:
# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


In [10]:
for i in range(10):
    print(len(x_train[i]), x_train[i][:4],'->', y_train[i])


218 [1, 14, 22, 16] -> 1
189 [1, 194, 1153, 194] -> 0
141 [1, 14, 47, 8] -> 0
550 [1, 4, 2, 2] -> 1
147 [1, 249, 1323, 7] -> 0
43 [1, 778, 128, 74] -> 0
123 [1, 2, 365, 1234] -> 1
562 [1, 4, 2, 716] -> 0
233 [1, 43, 188, 46] -> 1
130 [1, 14, 20, 47] -> 0


In [3]:
# Pad all sequences
padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

(25000,)
(25000,)


In [4]:
len(x_train[0]), y_train[0:10]

(218, array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64))

In [5]:
# Define the Keras model
model = Sequential()
model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
model.summary()

# Train the model
history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)

# Test the model after training
test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 15)           75000     
_________________________________________________________________
lstm (LSTM)                  (None, 10)                1040      
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 76,051
Trainable params: 76,051
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - ETA: 0s - loss: 0.6025 - accuracy: 0.6927

C:\Users\weiliu\Anaconda3\envs\dl\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


20000/20000 [==============================] - 12s 594us/sample - loss: 0.6025 - accuracy: 0.6927 - val_loss: 0.4644 - val_accuracy: 0.8196
Epoch 2/5
20000/20000 [==============================] - 12s 591us/sample - loss: 0.3938 - accuracy: 0.8528 - val_loss: 0.3493 - val_accuracy: 0.8648
Epoch 3/5
20000/20000 [==============================] - 12s 618us/sample - loss: 0.2947 - accuracy: 0.8923 - val_loss: 0.3488 - val_accuracy: 0.8592
Epoch 4/5
20000/20000 [==============================] - 13s 634us/sample - loss: 0.2484 - accuracy: 0.9105 - val_loss: 0.3295 - val_accuracy: 0.8702
Epoch 5/5
20000/20000 [==============================] - 13s 648us/sample - loss: 0.2143 - accuracy: 0.9273 - val_loss: 0.3651 - val_accuracy: 0.8636
Test results - Loss: 0.35858763427734375 - Accuracy: 86.46399974822998%
